In [6]:

const the_config = {
  hours_to_show: 5,
  refresh_interval: '$fn ({vars, hass, visible_range}) => "auto"',
  offset: '$fn ({vars, hass, visible_range}) => {vars.a_var = "from offset"; return 5;}',
  entities: [{
    entity: 'sensor.temperature',
    attribute: '$fn ({vars, hass, visible_range}) => "temperature"',
    offset: 5,
    unit_of_measurement: "W",
    text: "$fn ({vars, hass, visible_range, xs, ys}) => { vars.ys = ys; return ys.map(y => 'y=${y}')}",
    filters: [{
      add: '$fn ({vars, hass, visible_range}) => 5'
    }]
  }],
  defaults: {
    entity: {
      internal: '$fn ({vars, hass, visible_range, xs, ys}) => ys.map(y => y%2)',
      show_value: true,
    },
    yaxes: {
      autorange: '$fn ({vars, hass, visible_range}) => true',
      fixedrange: false,
    },
  },
  layout: {
    title: '$fn ({vars, hass, visible_range}) => visible_range',
  },
  disable_pinch_to_zoom: '$fn ({vars, hass, visible_range}) => vars.a_var',
  stuff_coming_from_fetched_data: '$fn ({vars, hass, visible_range}) => vars',
};

In [7]:
import merge from "lodash/merge";
import get from "lodash/get";
import has from "lodash/get";

async function fetchData(entityConfig, hours_to_show, globalOffset) {
  return {
    xs: [1, 2, 3, 4],
    ys: [21, 22, 23, 24],
    meta: {},
    // etc
  };
}

function isObjectOrArray(value) {
  return typeof value == "object" && !(value instanceof Date);
}
class ConfigParser {
  private input: {
    raw_config: any;
    visible_range: [number, number];
    hass: {};
  };
  private parsedConfig: {};
  private busy = true;
  public get config() {
    if (this.busy) throw new Error("busy");
    return this.parsedConfig;
  }
  private getFar(p: { path: string; callingPath: string }) {
    if (has(this.parsedConfig, p.path)) {
      return get(this.parsedConfig, p.path);
    }
    const value = get(this.input.raw_config, p.path);
    if (typeof value === "string" && value.startsWith("$fn")) {
      throw new Error(
        `Since [${p.path}] is a $fn, it has to be defined before [${p.callingPath}]`
      );
    }
    return value;
  }
  private async evalNode({
    parent,
    path,
    key,
    value,
    fnParam,
  }: {
    parent: object;
    path: string;
    key: string;
    value: any;
    fnParam: object;
  }) {
    if (path.match(/^defaults$/)) return;

    if (typeof value === "string" && value.startsWith("$fn")) {
      value = eval(value.slice(3));
    }
    if (typeof value === "function") value = value(fnParam);

    if (path.match(/^layout\.[xyz]axis$/)) {
      value = merge(value, this.input.raw_config.defaults?.axes, value);
    }
    if (path.match(/^entities\.\d+$/)) {
      if (typeof value === "string") value = { entity: value };
      value = merge(value, this.input.raw_config.defaults?.entity, value);

      const keys = Object.keys(value);
      const requiredKeys = [
        "entity",
        "offset",
        "statistic",
        "period",
        "attribute",
      ];
      const fetchAt = Math.max(...requiredKeys.map((k) => keys.indexOf(k)));
      const me: any = (parent[key] = {});
      for (let i = 0; i < keys.length; i++) {
        const childKey = keys[i];
        const childValue = value[childKey];
        if (i === fetchAt) {
          const hours_to_show = this.getFar({
            path: "hours_to_show",
            callingPath: path,
          });
          const globalOffset = this.getFar({
            path: "offset",
            callingPath: path,
          });
          const data = await fetchData(me, hours_to_show, globalOffset);
          me.x = data.xs;
          me.y = data.ys;
          fnParam = { ...fnParam, ...data };
        }
        await this.evalNode({
          parent: me,
          path: `${path}.${childKey}`,
          key: childKey,
          value: childValue,
          fnParam,
        });
      }
    } else if (isObjectOrArray(value)) {
      const me = (parent[key] = Array.isArray(value) ? [] : {});
      for (const [childKey, childValue] of Object.entries(value)) {
        await this.evalNode({
          parent: me,
          path: `${path}.${childKey}`,
          key: childKey,
          value: childValue,
          fnParam,
        });
      }
    } else {
      parent[key] = value;
    }
  }
  async update(input: {
    raw_config: Object;
    visible_range: [number, number];
    hass: {};
  }) {
    this.input = input;
    const fnParam = {
      vars: {},
      hass: input.hass,
      visible_range: input.visible_range,
    };
    this.busy = true;
    this.parsedConfig = {};
    for (const [key, value] of Object.entries(input.raw_config)) {
      await this.evalNode({
        parent: this.parsedConfig,
        path: key,
        key: key,
        value,
        fnParam,
      });
    }
    this.busy = false;
  }
}
const parser = new ConfigParser();
const visible_range: [number, number] = [2, 4];
const hass = {};
await parser.update({ raw_config: the_config, visible_range, hass });
console.log(JSON.stringify(parser.config, null, 2));


{
  "hours_to_show": 5,
  "refresh_interval": "auto",
  "offset": 5,
  "entities": [
    {
      "entity": "sensor.temperature",
      "attribute": "temperature",
      "x": [
        1,
        2,
        3,
        4
      ],
      "y": [
        21,
        22,
        23,
        24
      ],
      "offset": 5,
      "unit_of_measurement": "W",
      "text": [
        "y=${y}",
        "y=${y}",
        "y=${y}",
        "y=${y}"
      ],
      "filters": [
        {
          "add": 5
        }
      ],
      "internal": [
        1,
        0,
        1,
        0
      ],
      "show_value": true
    }
  ],
  "layout": {
    "title": [
      2,
      4
    ]
  },
  "disable_pinch_to_zoom": "from offset",
  "stuff_coming_from_fetched_data": {
    "a_var": "from offset",
    "ys": [
      21,
      22,
      23,
      24
    ]
  }
}
